In [8]:
import numpy as np 
import pandas as pd 
import geopandas as gpd 
from shapely.wkt import loads 
from shapely.geometry import Point

In [2]:
### extract highway links
edges = pd.read_csv('../edges_peer.csv')
edges.head(2)

,uniqueid,osmid,length,type,bridge,tunnel,lanes,maxspeed,capacity,fft,weight,start_nid,end_nid,start_sp,end_sp,geometry
0,0,7714200,175.637,residential,NaN,NaN,1,25.0,950.0,15.715577,18.858693,0,2,1,3,"LINESTRING (-122.7694487 38.4353358, -122.7692..."
1,1,7714200,62.870,residential,NaN,NaN,1,25.0,950.0,5.625457,6.750548,0,224222,1,224223,"LINESTRING (-122.7694487 38.4353358, -122.7695..."


In [43]:
edges['type'].unique()

array(['residential', 'secondary', 'tertiary', 'motorway', 'primary',
       'unclassified', 'secondary_link', 'motorway_link', 'living_street',
       'primary_link', 'road', 'trunk_link', 'trunk',
       "['unclassified', 'tertiary']", 'tertiary_link',
       "['residential', 'tertiary']", "['unclassified', 'residential']",
       "['road', 'residential']", "['secondary', 'residential']",
       "['motorway_link', 'secondary']",
       "['living_street', 'residential']", "['residential', 'secondary']",
       "['secondary', 'tertiary']", "['secondary', 'primary']",
       "['unclassified', 'secondary']", "['motorway_link', 'motorway']",
       "['motorway', 'trunk']", "['motorway_link', 'tertiary']",
       "['tertiary', 'primary']", "['trunk', 'primary']",
       "['unclassified', 'road']", "['primary', 'tertiary']",
       "['motorway_link', 'primary']", "['tertiary_link', 'residential']",
       "['primary', 'secondary', 'tertiary']",
       "['trunk_link', 'secondary']",
       "

In [45]:
# potential_edges = edges.loc[edges['maxspeed']>=40].reset_index(drop=True)
potential_edges = edges.loc[~edges['type'].isin(['residential', 'unclassified', 'road', 'living_street', "['unclassified', 'residential']"])].reset_index(drop=True)
potential_edges.to_csv('../edges_peer_bridge.csv')
potential_edges = gpd.GeoDataFrame(potential_edges, crs='epsg:4326', geometry=potential_edges['geometry'].map(loads)).to_crs('epsg:26910')
potential_edges['midpoint'] = potential_edges['geometry'].apply(lambda x: x.interpolate(0.5, normalized=True))
potential_edges['c_x'] = potential_edges['midpoint'].apply(lambda x: x.x)
potential_edges['c_y'] = potential_edges['midpoint'].apply(lambda x: x.y)
potential_edges.head(2)

,uniqueid,osmid,length,type,bridge,tunnel,lanes,maxspeed,capacity,fft,weight,start_nid,end_nid,start_sp,end_sp,geometry,midpoint,c_x,c_y
0,13,"[415868896, 415868937, 415868929]",337.456,secondary,NaN,NaN,1,25.0,950.0,30.194753,36.233704,4,218360,5,218361,"LINESTRING (608234.783 4131648.521, 608207.534...",POINT (608066.596 4131631.192),608066.596448,4.131631e+06
1,14,415868933,108.412,secondary,NaN,NaN,1,25.0,950.0,9.700446,11.640535,4,36113,5,36114,"LINESTRING (608234.783 4131648.521, 608240.666...",POINT (608288.776 4131654.430),608288.776247,4.131654e+06


In [7]:
### bridge data
bridges = pd.read_csv('BayArea_Bridge_DS4.csv')
bridges.head(2)

,new_id,Bridge_Number,Structure_Name,District,County,Route,Postmile,Location,Latitude,Longitude,...,damage_90,damage_91,damage_92,damage_93,damage_94,damage_95,damage_96,damage_97,damage_98,damage_99
0,1,37 0334,EAST DUNNE AVENUE OC,4,SCL,101,R16.01,04-SCL-101-R16.01-MGH,37.1308,-121.6331,...,0,0,0,0,0,0,0,0,0,0
1,2,37 0008L,CARNADERO CREEK,4,SCL,101,4.2,04-SCL-101-4.2,36.9761,-121.5556,...,0,0,0,0,0,0,0,0,0,0


In [11]:
bridges = gpd.GeoDataFrame(bridges, crs='epsg:4326', geometry=[Point(xy) for xy in zip(bridges['Longitude'], bridges['Latitude'])])

In [16]:
bridges = bridges.to_crs('epsg:26910')
bridges['b_x'] = bridges['geometry'].apply(lambda x: x.x)
bridges['b_y'] = bridges['geometry'].apply(lambda x: x.y)
bridges.head(2)

,new_id,Bridge_Number,Structure_Name,District,County,Route,Postmile,Location,Latitude,Longitude,...,damage_93,damage_94,damage_95,damage_96,damage_97,damage_98,damage_99,geometry,b_x,b_y
0,1,37 0334,EAST DUNNE AVENUE OC,4,SCL,101,R16.01,04-SCL-101-R16.01-MGH,37.1308,-121.6331,...,0,0,0,0,0,0,0,POINT (621416.054 4110256.889),621416.054359,4.110257e+06
1,2,37 0008L,CARNADERO CREEK,4,SCL,101,4.2,04-SCL-101-4.2,36.9761,-121.5556,...,0,0,0,0,0,0,0,POINT (628561.178 4093195.790),628561.177663,4.093196e+06


In [55]:
osmid_name_dict = {
    '202024214': 'Park Boulevard',
    '202024215': 'Park Boulevard',
    '416196599': 'Park Boulevard',
    '202024215': 'Park Boulevard',
    '516617056': 'Excelsior Avenue',
    '516617057': 'Excelsior Avenue',
    '37062614': 'Excelsior Avenue',
    '30013251': 'Chatham Road',
    '416196591': 'Chatham Road',
    '516617056': 'Excelsior Avenue',
    '516617057': 'Excelsior Avenue',
    '37062614': 'Excelsior Avenue',
    '202024214' : 'Park Boulevard',
    '518502217': 'MacArthur Boulevard'
}

In [58]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [62]:
for bridge_no in ['33 0326']:
    bridge_info = bridges.loc[bridges['Bridge_Number'] == bridge_no].iloc[0]
    print(bridge_info['b_x'], bridge_info['b_y'], bridge_info['Facility_Carried'])
    bridge_name = bridge_info['Facility_Carried'].replace('blvd', 'boulevard').replace(' NB', '')
    print(bridge_name)

    # distance to road links
    # print(np.column_stack([potential_edges['c_x'] - bridge_info['b_x'], potential_edges['c_y'] - bridge_info['b_y']]))
    bridge_road_distance = np.linalg.norm(
        np.column_stack([potential_edges['c_x'] - bridge_info['b_x'], potential_edges['c_y'] - bridge_info['b_y']]),
        axis=1)
    closest_links = potential_edges.loc[bridge_road_distance<200, :]
    # display(closest_links)
    for link in closest_links.itertuples():
        link_osmids = getattr(link, 'osmid').replace('[', '').replace(']', '').split(', ')
        for link_osmid in link_osmids:
            try:
                link_name = osmid_name_dict[link_osmid]
                print(link_osmid, link_name, similar(link_name.lower(), bridge_name.lower()))
            except KeyError:
                pass

567492.6307157357 4184456.389355603 MACARTHUR BLVD NB
MACARTHUR BLVD
202024214 Park Boulevard 0.5
202024215 Park Boulevard 0.5
416196599 Park Boulevard 0.5
416196599 Park Boulevard 0.5
202024215 Park Boulevard 0.5
516617056 Excelsior Avenue 0.26666666666666666
516617057 Excelsior Avenue 0.26666666666666666
37062614 Excelsior Avenue 0.26666666666666666
30013251 Chatham Road 0.46153846153846156
416196591 Chatham Road 0.46153846153846156
30013251 Chatham Road 0.46153846153846156
516617056 Excelsior Avenue 0.26666666666666666
516617057 Excelsior Avenue 0.26666666666666666
37062614 Excelsior Avenue 0.26666666666666666
202024214 Park Boulevard 0.5
518502217 MacArthur Boulevard 0.8484848484848485


In [63]:
import json

In [67]:
osm = json.load(open('target_tertiary_above.osm'))['elements']

In [82]:
osm_dict = dict()
for e in osm:
    if e['type'] == 'way':
        try:
            e_name = e['name']
        except KeyError:
            try:
                e_name = e['tags']['name']
            except KeyError:
                e_name = ''
        try:
            e_ref = e['tags']['ref']
        except KeyError:
            e_ref = ''
        osm_dict[str(e['id'])] = [e_ref, e_name]

In [120]:
osm_dict['479773091']

['US 101', 'Bayshore Freeway']

In [121]:
bridge_linkid_df_list = []
for bridge_info in bridges.itertuples():
    # print('\n')
    bridge_name = getattr(bridge_info, 'Facility_Carried').lower()
    bridge_name = bridge_name.replace('nb', '').replace('sb', '').replace('wb', '').replace('eb', '')
    bridge_name = bridge_name.replace('blvd', 'boulevard').replace('ave', 'avenue').replace('highway', '')
    bridge_name = bridge_name.replace('state route', 'ca').replace('interstate', 'ca').replace('route ', 'i').replace('hwy', '').replace('highway', '')
    bridge_name = bridge_name.replace('u.s.', 'us')
    
    bridge_no = getattr(bridge_info, 'Bridge_Number')
    bridge_bx = getattr(bridge_info, 'b_x')
    bridge_by = getattr(bridge_info, 'b_y')
    # print(bridge_name, bridge_no)

    # distance to road links
    bridge_road_distance = np.linalg.norm(
        np.column_stack([potential_edges['c_x'] - bridge_bx, potential_edges['c_y'] - bridge_by]),
        axis=1)
    closest_links = potential_edges.loc[bridge_road_distance<200, :]
    closest_links['distance_to_bridge'] = bridge_road_distance[bridge_road_distance<200]
    candidate_links = []
    largest_name_similarity, second_largest_name_similarity = 0.7, 0.7
    for link in closest_links.itertuples():
        link_osmids = getattr(link, 'osmid').replace('[', '').replace(']', '').split(', ')
        # print(link_osmids)
        for link_osmid in link_osmids:
            try:
                link_name_list = osm_dict[link_osmid]
                if link_name_list[0] != '':
                    link_name = link_name_list[0].lower()
                else:
                    link_name = link_name_list[1].lower()
                name_similarity = similar(link_name, bridge_name)
                link_bridge_distance = getattr(link, 'distance_to_bridge')
                # print(getattr(link, 'uniqueid'), link_osmid, link_name_list, name_similarity)
            except KeyError:
                pass
            
            if name_similarity>=largest_name_similarity:
                largest_name_similarity = name_similarity
                candidate_links.append([getattr(link, 'uniqueid'), name_similarity, link_bridge_distance, bridge_no, bridge_bx, bridge_by])
            elif name_similarity>=second_largest_name_similarity:
                second_largest_name_similarity = name_similarity
                candidate_links.append([getattr(link, 'uniqueid'), name_similarity, link_bridge_distance, bridge_no, bridge_bx, bridge_by])
            else:
                pass
    if len(candidate_links) == 0:
        continue
    candidate_links_df = pd.DataFrame(candidate_links, columns=['link_id', 'name_similarity', 'link_bridge_distance', 'bridge_no', 'bridge_bx', 'bridge_by'])
    candidate_links_df = candidate_links_df.groupby('link_id').agg({'name_similarity': np.mean, 'link_bridge_distance': np.min, 'bridge_no': 'first', 'bridge_bx': 'first', 'bridge_by': 'first'}).reset_index()
    bridge_linkid_df_list.append(candidate_links_df)
bridge_linkid_df = pd.concat(bridge_linkid_df_list)
bridge_linkid_df = bridge_linkid_df.merge(potential_edges[['uniqueid', 'c_x', 'c_y']], left_on='link_id', right_on='uniqueid')
bridge_linkid_df['geometry'] = bridge_linkid_df.apply(lambda x: 'LINESTRING ({} {}, {} {})'.format(x['bridge_bx'], x['bridge_by'], x['c_x'], x['c_y']), axis=1)
bridge_linkid_df.to_csv('caltrans_bridge_to_osm_links.csv', index=False)

In [122]:
len(bridge_linkid_df['bridge_no'].unique())

714

In [118]:
len(bridges['Bridge_Number'].unique())

1742

In [123]:
bridges.loc[~bridges['Bridge_Number'].isin(bridge_linkid_df['bridge_no'].unique()), 'Facility_Carried'].unique()

array(['SB US HIGHWAY 101', 'NB U.S. HWY 101', 'U.S. HIGHWAY 101',
       'GREAT AMERICA-W237', 'STATE ROUTE 17', 'U.S. HWY 101 (NB)',
       'ON-RAMP EB RTE 152', 'WESTFIELD-DOWNING', 'STATE ROUTE 152',
       'SB STATE ROUTE 87', 'U.S. HWY 101 (SB)', 'STATE ROUTE 130',
       'EB STATE ROUTE 152', 'ROUTE 152(10TH ST)', 'S87- N&S 280 CONN',
       'NB US HIGHWAY 101', 'SUNSET POC', 'STATE ROUTE 82',
       'TRIMBLE ROAD POC', 'OREGON EXPRESSWAY', 'SB STATE ROUTE 82',
       'NB STATE ROUTE 85', 'STATE ROUTE 85 NB', 'SB STATE ROUTE 85',
       'N101-N85 CONNECTOR', 'WB STATE ROUTE 152', 'N. SHORELINE BLVD',
       'BLUE HILLS POC', 'DENT AVENUE POC', 'SAMARITAN PLC POC',
       'UNION AVENUE', 'BASCOM AVE ON RAMP', 'N17 ON RAMP',
       'S87-S85 CONNECTOR', '87,280,680 ON-RAMP', 'S17-S85 CONNECTOR',
       'SB 85 OFF-RAMP', 'N17-N85 CONNECTOR', 'COX AVE UTILITY OC',
       'N85-N87 CONNECTOR', 'S87 - SANTA TERESA', 'S17-S85 CONN UC',
       'UP RR', 'NB STATE ROUTE 82', 'N17-N85 & S85-